# Data Exploration Github Data

#### Read the Dataframe

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# read the data
df = pd.read_csv("../data/total_transformed.csv",
                 index_col='date', 
                 parse_dates=['date'],
                 infer_datetime_format=True)

/home/geanderson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,6,7,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# checking the data shape
df.outside_contribution.value_counts()

1.0    322530
0.0    278547
Name: outside_contribution, dtype: int64

#### Delete some columns from my dataframe

In [6]:
# dropping the columns that we do not need
df.drop('sha', axis=1, inplace=True)
df.drop('message', axis=1, inplace=True)
df.drop('status', axis=1, inplace=True)
df.drop('name', axis=1, inplace=True)
df.drop('fullname', axis=1, inplace=True)
df.drop('filenames', axis=1, inplace=True)
df.drop('comments', axis=1, inplace=True)
df.drop('additions', axis=1, inplace=True)
df.drop('deletions', axis=1, inplace=True)
df.drop('changes', axis=1, inplace=True)
df.drop('owner', axis=1, inplace=True)
df.drop('open_issues_count', axis=1, inplace=True)
df.drop('forks_count', axis=1, inplace=True)
df.drop('watchers_count', axis=1, inplace=True)
df.drop('date_only', axis=1, inplace=True)
df.drop('variance_commits', axis=1, inplace=True)
df.drop('results', axis=1, inplace=True)

In [7]:
# save the csv
df.to_csv('../data/total_dry.csv')